In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Pair Programming Decision Tree

En el pair programming de hoy debéis usar el csv que guardastéis cuando hicistéis el pairprgramming de codificicación (este csv debería tener las variables estadandarizas).

En pairprogramming anteriores ajustastéis vuestro datos a una regresión lineal. El objetivo de hoy es:
- Ajustar el modelo a un Decision Tree
- Extraer las métricas
- Debatid entre vosotras que modelo es mejor y por qué (basándose en las métricas)

In [3]:
df = pd.read_csv('datos/soci_econ_country_profiles_v7.csv', index_col=0)
df.head()

,Region,GDP_per_capita,Employment_Agriculture,Employment_Services,Agricultural_production_index,Urban_population,Urban_population_growth_rate,Fertility_rate_total,Infant_mortality_rate,Seats_held_by_women_in_national_parliaments,Individuals_using_the_Internet,Energy_production_primary,Energy_supply_per_capita,Quality_Of_Life_Index,Purchasing_Power_Index,Safety_Index,Health_Care_Index,Property_price_to_income_ratio,Traffic_commute_time_index,Pollution_index,Climate_index,Affordability_Index,Cost_Of_Living_Index,Consumer_price_index,Current_health_expenditure,Human_capital_index,Inflation_consumer_prices,Life_expectancy_at_birth_total,Labour_force_participation,Pop_using_improved_drinking_water_urban,Pop_using_improved_drinking_water_rural,Population_age_distribution_0_14,Population_age_distribution_60_mas,Education_Secondary_gross_enrol_ratio,Education_Tertiary_gross_enrol_ratio
country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Argentina,1.84,-0.431499,-0.793293,0.635685,0.451515,1.176545,-0.146278,0.816818,0.390527,1.373669,-0.170271,-0.433123,-0.072257,139.59,-0.480096,-1.843490,0.739348,0.085614,0.253993,0.085118,1.031920,-1.242902,-0.124619,-0.375346,0.595817,-0.661735,-0.179334,-0.356054,0.526225,0.213117,0.576120,0.983668,-0.481642,0.119029,0.928674
Australia,2.30,1.358804,-0.729591,0.874487,-0.051041,1.039907,0.271660,0.093270,-0.510644,0.436646,1.674834,0.729295,0.579071,176.54,1.024540,-0.270524,0.839394,-0.461542,-0.207625,-1.254411,0.556493,0.080872,1.230619,-0.300239,0.626171,1.061166,-0.270570,1.069555,0.052782,0.675804,0.681506,-0.044663,0.256053,1.643795,1.272043
Austria,2.30,1.006731,-0.547584,0.365043,-0.239499,-0.292313,-0.647804,-0.811165,-0.565818,0.611189,-0.538225,0.186268,0.579071,190.37,0.807519,1.532822,1.540764,-0.380339,-1.420448,-1.379278,0.273045,0.414798,0.789648,-0.345498,1.068892,0.971432,-0.238703,0.870393,-0.574209,0.675804,0.681506,-0.898700,0.796151,-0.222299,0.863591
Belarus,1.84,-0.860425,-0.101668,-0.534443,0.639974,0.316865,-0.982155,-0.449391,-0.538231,0.969463,-0.786194,-0.119185,0.513938,119.23,-1.239326,1.298550,-1.318426,1.373269,-1.221178,-0.552731,-0.845325,-1.397939,-1.028272,1.151509,-0.593477,-0.907608,0.710551,-0.877799,-0.318294,-0.042578,0.383944,-0.445537,0.295572,0.131486,1.157587
Belgium,2.30,0.819859,-0.866096,0.993887,-0.239499,1.523832,-0.564217,-0.087617,-0.547427,1.290990,-0.754198,0.508691,0.579071,160.52,0.647171,-0.433191,1.331195,-0.902361,0.011715,-0.120354,0.619511,1.070722,0.974914,-0.368860,1.047345,0.648387,-0.227962,0.835214,-1.457115,0.614924,0.644311,-0.375820,0.730285,3.118732,0.569595


In [4]:
# separamos los datos en X (VP) y y (VR)

X = df.drop(['Quality_Of_Life_Index'], axis=1)
y = df ['Quality_Of_Life_Index']

In [5]:
# dividimos los datos en datos de entrenamiento y datos de prueba

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 37)

In [6]:
# ajustamos el modelo

arbol = DecisionTreeRegressor(random_state=37)
arbol.fit(x_train, y_train)

DecisionTreeRegressor(random_state=37)

In [7]:
# hacemos las predicciones

y_pred_test_dt = arbol.predict(x_test)
y_pred_train_dt = arbol.predict(x_train)

In [8]:
# función para sacar las métricas a un dataframe

def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [9]:
# sacamos un dataframe de los resultados

train_df = pd.DataFrame({'Real': y_train, 'Predicted': y_pred_train_dt, 'Set': ['Train']*len(y_train)})
test_df = pd.DataFrame({'Real': y_test, 'Predicted': y_pred_test_dt, 'Set': ['Test']*len(y_test)})
resultados = pd.concat([train_df,test_df], axis = 0)

In [10]:
# sacamos un dataframe de las metricas 

dt_results1 = metricas(y_test, y_train, y_pred_test_dt, y_pred_train_dt, "Decision Tree I")
dt_results1

,MAE,MSE,RMSE,R2,set,modelo
0,17.704615,794.803877,28.192266,0.361354,test,Decision Tree I
1,0.000000,0.000000,0.000000,1.000000,train,Decision Tree I


In [11]:
# calculamos el max_features que ha usado el modelo por defecto

max_features = np.sqrt(len(x_train.columns))
max_features

5.830951894845301

In [12]:
# calculamos el max_depth que ha usado el modelo por defecto

print(arbol.tree_.max_depth)

9


In [13]:
# creamos el diccionario de hiperparametros para usar con GridSearch para encontrar el mejor modelo

param = {"max_depth": [2,4,6], 
        "max_features": [1,2,3,4],
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]} 

In [14]:
# creamos la instancia

gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param, 
            cv=10,  
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [15]:
# ajustamos el modelo

gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 4, 6], 'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [16]:
# sacamos los resultados del best_estimator_

mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=4, max_features=2, min_samples_leaf=10,
                      min_samples_split=10)

In [17]:
# refinamos los hiperparametros

param2 = {"max_depth": [3,4,5], 
        "max_features": [3,4,5],
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]}

In [18]:
# creamos una nueva instancia del GridSearch

gs2 = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param2, 
            cv=10,  
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [19]:
# volvemos a ajustar el modelo

gs2.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [3, 4, 5], 'max_features': [3, 4, 5],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [20]:
# sacamos los resultados del best_estimator_ con el nuevo modelo

mejor_modelo2 = gs2.best_estimator_
mejor_modelo2

DecisionTreeRegressor(max_depth=4, max_features=4, min_samples_leaf=10,
                      min_samples_split=10)

In [21]:
# volvemos a refinar los parametros

param3 = {"max_depth": [3,4,5], 
        "max_features": [3,4,5],
        "min_samples_split": [5, 10, 20],
        "min_samples_leaf": [5, 10, 20]}

In [22]:
# creamos una nueva instancia

gs3 = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param3, 
            cv=10,  
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error")

In [23]:
# lo ajustamos al modelo

gs3.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [3, 4, 5], 'max_features': [3, 4, 5],
                         'min_samples_leaf': [5, 10, 20],
                         'min_samples_split': [5, 10, 20]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [24]:
# volvemos a sacar los resultados del best_estimator_ 
mejor_modelo3 = gs3.best_estimator_
mejor_modelo3

DecisionTreeRegressor(max_depth=5, max_features=5, min_samples_leaf=10,
                      min_samples_split=10)

In [25]:
# nos quedamos con los ultimos resultados del best_estimator_, asi que volvemos a sacar las predicciones con 
# este tercer modelo

y_pred_test_dt2 = mejor_modelo3.predict(x_test)
y_pred_train_dt2 = mejor_modelo3.predict(x_train)

In [27]:
# sacamos un dataframe de las metricas

dt_results2 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "Decision tree II")
dt_results2


,MAE,MSE,RMSE,R2,set,modelo
0,16.139883,803.812824,28.351593,0.354115,test,Decision tree II
1,11.612891,270.326345,16.441604,0.825574,train,Decision tree II


In [28]:
# sacamos un dataframe de los resultados

train_df2 = pd.DataFrame({'Real': y_train, 'Predicted': y_pred_train_dt2, 'Set': ['Train']*len(y_train)})
test_df2 = pd.DataFrame({'Real': y_test, 'Predicted': y_pred_test_dt2, 'Set': ['Test']*len(y_test)})
resultados2 = pd.concat([train_df2,test_df2], axis = 0)

In [29]:
resultados2

,Real,Predicted,Set
country,,,
Romania,143.04,135.715455,Train
Egypt,91.81,90.189412,Train
New Zealand,184.74,177.050000,Train
Belgium,160.52,167.317692,Train
Vietnam,36.12,90.189412,Train
Colombia,115.38,90.189412,Train
Lithuania,130.28,135.715455,Train
Sweden,172.74,167.317692,Train
Germany,189.74,167.317692,Train


In [30]:
# cargamos los resultados del model de regresion linear

df_linear_results = pd.read_csv("datos/resultados_RL.csv", index_col = 0)
df_linear_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.010049,0.000130,0.011398,1.0,test,Linear Regresion
1,0.001999,0.000008,0.002763,1.0,train,LinearRegression


In [31]:
# concatenamos los resultados de los tres modelos para compararlos

df_DT_LR_results = pd.concat([df_linear_results, dt_results1, dt_results2], axis = 0)
df_DT_LR_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.010049,0.000130,0.011398,1.000000,test,Linear Regresion
1,0.001999,0.000008,0.002763,1.000000,train,LinearRegression
0,17.704615,794.803877,28.192266,0.361354,test,Decision Tree I
1,0.000000,0.000000,0.000000,1.000000,train,Decision Tree I
0,16.139883,803.812824,28.351593,0.354115,test,Decision tree II
1,11.612891,270.326345,16.441604,0.825574,train,Decision tree II


Aquí vemos los resultados de los tres modelos que hemos hecho:
- regresión lineal
- decision tree 1, antes de ajustar los hiperparámetros
- decision tree 2, después de ajustar los hiperparámetros

Podemos concluir que con un R2 de 1.0, para la regresión lineal tuvimos un caso de overfitting. Para el decision tree 1, también hemos tenido overfitting, ya que las métricas del grupo de entrenamiento son mucho mejores que las del grupo test y el R2 del grupo train es 1.0. Entonces, un nuestra opinion los mejores resultados son los del decision tree 2, aunque aún vemos overfitting, pero es posible que no se puede evitar con tan pocos datos.

In [32]:
df_DT_LR_results.to_csv("datos/resultados_RL.csv")